### Validate PhoSim centroid file

This notebook will read in a PhoSim InstanceCatalog and the corresponding centroid file, use CatSim to calculate the pixel positions of each source in the InstanceCatalog, and then compare those predicted positions with the pixel positions reported by PhoSim.  Note: since CatSim does not rigorously treat atmospheric refraction, we should expect there to be some offset between the CatSim- and PhoSim-reported pixel positions.

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
import os
data_dir = os.path.join("/Users", "danielsf", "physics",
                        "lsst_160212", "Development",
                        "Twinkles", "workspace")

cat_name = os.path.join(data_dir,
                       "phosim_231488_cat.txt")

centroid_name = os.path.join(data_dir,
                             "centroid_lsst_e_231488_f5_R22_S11_E000.txt")

In [ ]:
from desc.twinkles import getPredictedCentroids
catsim_data = getPredictedCentroids(cat_name)

In [ ]:
import pandas as pd
import numpy as np

dtype = np.dtype([('id',long), ('nphot', int), ('x',float), ('y',float)])
_phosim_data = np.genfromtxt(centroid_name, dtype=dtype, skip_header=1)
phosim_data = pd.DataFrame({'id': _phosim_data['id'],
                            'nphot': _phosim_data['nphot'],
                            'x': _phosim_data['x'],
                            'y': _phosim_data['y']})


Make sure that any sources which appear in the PhoSim centroid file, but not in the CatSim-predicted centroid file have `id==0` (this is actually paranoid, since we have calculated the pixel positions of all of the objects in the catalog as if they did land on the Twinkles chip)

In [ ]:
just_phosim = phosim_data[np.logical_not(phosim_data.id.isin(catsim_data.id.values).values)]
print len(just_phosim)
try:
    assert just_phosim.id.max() == 0
except:
    print 'a source with non-zero ID appears in PhoSim centroid file, but not CatSim centroid file'
    raise


Find all of the CatSim sources that appeared in the PhoSim centroid file

In [ ]:
catsim_phosim = catsim_data.merge(phosim_data, left_on='id', right_on='id',
                                  how='left', suffixes=('_catsim', '_phosim'))

catsim_phosim['dx'] = pandas.Series(catsim_phosim['x_catsim']-catsim_phosim['x_phosim'], index=catsim_phosim.index)
catsim_phosim['dy'] = pandas.Series(catsim_phosim['y_catsim']-catsim_phosim['y_phosim'], index=catsim_phosim.index)

# select points that actually showed up with non-NaN positions on the PhoSim image
overlap = np.logical_not(np.logical_or(catsim_phosim['x_phosim'].isnull(),
                                    catsim_phosim['y_phosim'].isnull()))

overlap = catsim_phosim[overlap]

bright_sources = overlap.query('nphot>0')
bright_sources = bright_sources.sort_values(by='nphot')


Plot the displacement between CatSim and PhoSim position in pixels.  The color bar indicates the number of photons in each source.  We will do three plots, at different degrees of resolution for the InstanceCatalog.

In [ ]:
for i_fig, limit in enumerate(((-20, 20), (-50, 50), (-200,200))):
    plt.figure()
    plt.scatter(bright_sources.dx,bright_sources.dy,c=bright_sources.nphot,
                s=10,edgecolor='',cmap=plt.cm.gist_ncar,norm=LogNorm())

    ticks = np.arange(limit[0],limit[1],(limit[1]-limit[0])/6)
    tick_labels = ['%.2e' % vv if ix%2==0 else '' for ix, vv in enumerate(ticks)]

    plt.xlim(limit)
    plt.ylim(limit)
    plt.xticks(ticks, tick_labels, fontsize=10)
    plt.yticks(ticks, tick_labels, fontsize=10)
    plt.minorticks_on()
    plt.tick_params(axis='both', length=10)

    cb = plt.colorbar()
    cb.set_label('photons in source')
    plt.xlabel('dx (pixels)')
    plt.ylabel('dy (pixels)')

plt.figure()
plt.scatter(bright_sources.dx,bright_sources.dy,c=bright_sources.nphot,
            s=10,edgecolor='',cmap=plt.cm.gist_ncar,norm=LogNorm())

plt.minorticks_on()
plt.tick_params(axis='both', length=10)

cb = plt.colorbar()
cb.set_label('photons in source')
plt.xlabel('dx (pixels)')
plt.ylabel('dy (pixels)')


Now plot the maximum absolute displacement in x and y as a function of the number of photons (i.e. for each unique value of `nphot`, plot the maximum of the absolute value of `dx` and `dy` for all sources with `nphot` greater than or equal to that unique value).  Again: show the same plot at different resolutions.

In [ ]:
nphot_unique = np.unique(bright_sources.nphot)
sorted_dex = np.argsort(-1.0*nphot_unique)
nphot_unique = nphot_unique[sorted_dex]

dx_of_nphot = np.array([np.abs(bright_sources.query('nphot>=%e' % nn).dx).max() for nn in nphot_unique[1:]])
dy_of_nphot = np.array([np.abs(bright_sources.query('nphot>=%e' % nn).dy).max() for nn in nphot_unique[1:]])

true_max = max(dx_of_nphot.max(), dy_of_nphot.max())

plt.figsize = (30, 30)
for ymax in (true_max, 200, 100, 50):
    plt.figure()
    plt.semilogx(nphot_unique[1:], dx_of_nphot, label='dx')
    plt.semilogx(nphot_unique[1:], dy_of_nphot, label='dy')
    plt.xlabel('minimum number of photons')
    plt.ylabel('max pixel displacement')
    plt.legend(loc=0)
    plt.ylim(0,ymax)


Calculate the weighted mean, mean, and median displacements in `x` and `y`

In [ ]:
nphot_sum = bright_sources.nphot.sum()
weighted_dx = (bright_sources.dx*bright_sources.nphot).sum()/nphot_sum
weighted_dy = (bright_sources.dy*bright_sources.nphot).sum()/nphot_sum

print 'weighted dx (dy): ',weighted_dx, weighted_dy
print 'mean dx (dy): ',bright_sources.dx.mean(),bright_sources.dy.mean()
print 'median dx (dy): ',bright_sources.dx.median(),bright_sources.dy.median()
